# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1524041113502_0012,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
spark = SparkSession.builder.appName('customer_churn_project').getOrCreate()

In [3]:
df = spark.read.csv('hdfs://...customer_churn.csv',
                    inferSchema=True,header=True)

In [4]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+--------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|        Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+--------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:...|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:...|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:...|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:...|13120 Daniel Moun...|           Smith Inc|    1|

In [5]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)

In [6]:
df.columns

['Names', 'Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites', 'Onboard_date', 'Location', 'Company', 'Churn']

In [7]:
# Check to see if there are any missing values. Since each category has a count of 900, there are no missing values.

df.describe().show()


# if you were to drop na values you would do code below. However, maybe better to fill in 
# values such as with column averages

# final_df = df.na.drop()#.show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [8]:
(training_set, test_set) = df.randomSplit([0.7, 0.3])

# Pipeline Prep: Categorical data to boolean matrix, assemble feature vector, and make model

#### 2 steps (index then one hot encode)

In [9]:
from pyspark.ml.feature import (OneHotEncoder, StringIndexer)

In [10]:
# 1 string index and then 2 one hot encode. 
# turns catebories into indexes like 1,2,3
# one hot encode turns each index into a boolean matrix so only 0's and 1's

location_indexer = StringIndexer(inputCol = "Location", outputCol= "LocationIndex")
location_encoder = OneHotEncoder(inputCol="LocationIndex", outputCol="locationvector")

In [11]:
company_indexer = StringIndexer(inputCol = "Company", outputCol= "CompanyIndex")
company_encoder = OneHotEncoder(inputCol="CompanyIndex", outputCol="companyvector")

#### Assemble Feature Vector

In [12]:
# build features vector (leave out label which in this case is "churn")
# we only keep columns that we want to use as features for our ML model

from pyspark.ml.feature import (VectorAssembler, VectorIndexer)

assembler = VectorAssembler(inputCols=["Age",
                                       "Total_Purchase",
                                       "Years",
                                       "Num_Sites",
                                       "locationvector",
                                       "companyvector"], outputCol="features")

#### Make Model 

In [13]:
from pyspark.ml.classification import LogisticRegression

In [14]:
logistic_reg_churn = LogisticRegression(featuresCol="features", labelCol="Churn")

# Pipeline: making workflow

In [15]:
from pyspark.ml import Pipeline

In [16]:
pipeline = Pipeline(stages = [location_indexer, 
                              company_indexer,
                              location_encoder,
                              company_encoder,
                              assembler,
                              logistic_reg_churn])

In [17]:
# a column called "predictions" will be created when you do this
# you are fitting to training set and then applying that newly made model to the test set
fitted_model_to_training_set = pipeline.fit(training_set)

results = fitted_model_to_training_set.transform(test_set)

In [18]:
###

# ERROR, maybe because you "locations" and "company" are NOT really categories. Almost all of them are unique!
results.select("Churn", "prediction").show()

An error occurred while calling o225.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 45.0 failed 4 times, most recent failure: Lost task 0.3 in stage 45.0 (TID 49, wn2-yhphil.ourta4v5mcau1gb0wpnqcg2qtb.fx.internal.cloudapp.net, executor 1): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:377)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:234)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:228)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$25.apply(RDD.scala:827)
	at org.apache.spark.r

# Something is wrong. Try taking "location" and "company" out since most of them are NOT unique anyway.

In [20]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer)

assembler_revised = VectorAssembler(inputCols=["Age",
                                               "Total_Purchase",
                                               "Years",
                                               "Num_Sites"], outputCol="features")

In [22]:
from pyspark.ml import Pipeline

pipeline_revised = Pipeline(stages = [assembler_revised, logistic_reg_churn])

In [23]:
# train on training_set and apply to test set

trained_model = pipeline_revised.fit(training_set)

results_revised = trained_model.transform(test_set)

In [24]:
results_revised.select("Churn", "probability", "prediction").show()

+-----+--------------------+----------+
|Churn|         probability|prediction|
+-----+--------------------+----------+
|    0|[0.99743276331492...|       0.0|
|    0|[0.91271280232428...|       0.0|
|    0|[0.97940753647492...|       0.0|
|    0|[0.98873180213160...|       0.0|
|    0|[0.33926499966210...|       1.0|
|    0|[0.78679030003224...|       0.0|
|    0|[0.92445602289555...|       0.0|
|    0|[0.99282424486168...|       0.0|
|    1|[0.65837238759989...|       0.0|
|    0|[0.93879882893471...|       0.0|
|    0|[0.55007777911216...|       0.0|
|    0|[0.98930496405472...|       0.0|
|    0|[0.95317477930104...|       0.0|
|    0|[0.60449203169551...|       0.0|
|    1|[0.84797385831219...|       0.0|
|    0|[0.99306649752273...|       0.0|
|    0|[0.94846416196978...|       0.0|
|    0|[0.97637470604408...|       0.0|
|    1|[0.12430520506404...|       1.0|
|    0|[0.98934070130561...|       0.0|
+-----+--------------------+----------+
only showing top 20 rows

# Evaluating Results_Revised

In [25]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, 
                                   MulticlassClassificationEvaluator)

In [26]:
evaluation = BinaryClassificationEvaluator(rawPredictionCol="prediction",
                                          labelCol="Churn")

In [27]:
# Area Under Curve evaluating

AUC = evaluation.evaluate(results_revised)

In [28]:
AUC

0.7026932318782539

# Test Model on New Customers

### This is more like a real test set. The previous "test" set is more like a Cross-Validation set

In [29]:
re_test_df = spark.read.csv("wasb:///sparkcourse/data/new_customers.csv",
                            inferSchema = True, header = True)

In [30]:
# NOTICE THAT THERE IS NO CHURN COLUMN

re_test_df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)

In [31]:
# apparently, no need to select only the features you are working with for the model to work. 
# You can provide the whole dataset and the model will only use what it was trained to use and it
# will also make a "feature" vector. So, you don't need to assemble a feature vector (see table below)

new_customer_results = trained_model.transform(re_test_df)

In [32]:
new_customer_results.show()

+--------------+----+--------------+---------------+-----+---------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|        Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:...|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,7.7...|[2.44641938853100...|[0.92029921300875...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:...|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,9.2...|[-5.4186041014652...|[0.00441376313846...|  

In [33]:
new_customer_results.select("Company", "probability", "prediction").show()

+----------------+--------------------+----------+
|         Company|         probability|prediction|
+----------------+--------------------+----------+
|        King Ltd|[0.92029921300875...|       0.0|
|   Cannon-Benson|[0.00441376313846...|       1.0|
|Barron-Robertson|[0.04937691283449...|       1.0|
|   Sexton-Golden|[0.00909841809851...|       1.0|
|        Wood LLC|[0.80426833547650...|       0.0|
|   Parks-Robbins|[0.17447293780169...|       1.0|
+----------------+--------------------+----------+

In [35]:
new_customer_results.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+------------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|        prediction|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+------------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|                 6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|0.6666666666666666|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|              

#  Client Companies likely to churn from new customers list according to logistic regression model: Cannon-Benson, Barron, Barron-Robertson, Sexton-Golden, and Park-Robbins.